In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [3]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [4]:
import json
import os
from pathlib import Path
from collections import defaultdict, Counter
import pandas as pd
import hashlib
import glob

In [ ]:
def analyze_total_categories_from_annotations():
    """train_annotations에서 전체 카테고리 정보 추출"""
    print("=== 1단계: 전체 카테고리 82개 확인 ===")

    # 어노테이션 폴더 경로
    annotations_root = "./data/train_annotations"

    if not os.path.exists(annotations_root):
        print(f"❌ 어노테이션 폴더가 없습니다: {annotations_root}")
        return None

    # 모든 카테고리 정보 수집
    all_category_ids = set()
    category_id_to_name = {}
    category_counts = defaultdict(int)
    total_files = 0
    processed_files = 0

    print(f"📁 어노테이션 폴더 탐색: {annotations_root}")

    # 모든 하위 폴더 순회
    for root, dirs, files in os.walk(annotations_root):
        for file in files:
            if file.endswith('.json'):
                total_files += 1
                file_path = os.path.join(root, file)

                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                    # categories 정보 추출
                    if 'categories' in data:
                        for category in data['categories']:
                            cat_id = category['id']
                            cat_name = category.get('name', f'category_{cat_id}')

                            all_category_ids.add(cat_id)
                            category_id_to_name[cat_id] = cat_name
                            category_counts[cat_id] += 1

                    processed_files += 1

                    if processed_files % 100 == 0:
                        print(f"  처리 중: {processed_files}/{total_files} 파일")

                except Exception as e:
                    print(f"  ⚠️ 파일 읽기 실패: {file_path} - {e}")

    # 결과 정리
    sorted_category_ids = sorted(all_category_ids)

    print(f"\n=== 분석 결과 ===")
    print(f"📄 전체 JSON 파일 수: {total_files}")
    print(f"✅ 처리된 파일 수: {processed_files}")
    print(f"🎯 발견된 고유 카테고리 수: {len(all_category_ids)}개")

    if len(all_category_ids) > 0:
        print(f"📊 카테고리 ID 범위: {sorted_category_ids[0]} ~ {sorted_category_ids[-1]}")
        print(f"📋 카테고리 ID들 (처음 15개): {sorted_category_ids[:15]}")

        # 82개 확인
        if len(all_category_ids) == 82:
            print("✅ 예상한 82개 카테고리와 정확히 일치!")
        else:
            print(f"⚠️ 예상과 다름: {len(all_category_ids)}개 (예상: 82개)")
            if len(all_category_ids) < 82:
                print(f"   누락된 카테고리: {82 - len(all_category_ids)}개")
            else:
                print(f"   추가 카테고리: {len(all_category_ids) - 82}개")

    # 샘플 카테고리 정보 출력
    print(f"\n=== 카테고리 샘플 (처음 10개) ===")
    for i, cat_id in enumerate(sorted_category_ids[:10]):
        cat_name = category_id_to_name.get(cat_id, 'Unknown')
        count = category_counts[cat_id]
        print(f"  {i+1:2d}. ID: {cat_id:5d} | Name: {cat_name:30s} | 출현: {count:3d}회")

    # 카테고리 출현 빈도 분석
    print(f"\n=== 카테고리 출현 빈도 통계 ===")
    counts = list(category_counts.values())
    print(f"평균 출현 횟수: {sum(counts)/len(counts):.1f}")
    print(f"최소 출현 횟수: {min(counts)}")
    print(f"최대 출현 횟수: {max(counts)}")

    # 희귀 카테고리 (출현 횟수가 적은) 찾기
    rare_threshold = 5
    rare_categories = [cat_id for cat_id, count in category_counts.items() if count <= rare_threshold]
    if rare_categories:
        print(f"\n=== 희귀 카테고리 ({rare_threshold}회 이하 출현) ===")
        print(f"희귀 카테고리 수: {len(rare_categories)}개")
        for cat_id in sorted(rare_categories)[:10]:
            count = category_counts[cat_id]
            name = category_id_to_name.get(cat_id, 'Unknown')
            print(f"  ID: {cat_id:5d} | {name:30s} | {count}회")

    # 결과 저장
    result = {
        'total_categories': len(all_category_ids),
        'category_ids': sorted_category_ids,
        'category_mapping': category_id_to_name,
        'category_counts': dict(category_counts),
        'rare_categories': sorted(rare_categories) if rare_categories else []
    }

    # JSON으로 저장
    output_file = 'total_categories_analysis.json'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print(f"\n💾 결과 저장: {output_file}")

    return result

In [ ]:
def verify_with_alternative_methods():
    """대안 방법들로 카테고리 수 검증"""
    print(f"\n{'='*50}")
    print("=== 대안 방법으로 검증 ===")

    alternative_files = [
        './data/yolo_format/category_id_mapping.json',
        './data/processed/category_id_mapping.json',
        './data/categories.json',
        './runs/detect/pill_rtdetr/args.yaml'
    ]

    for file_path in alternative_files:
        if os.path.exists(file_path):
            print(f"\n📄 파일 발견: {file_path}")
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    if file_path.endswith('.json'):
                        data = json.load(f)

                        # 다양한 카테고리 필드 확인
                        category_fields = ['categories', 'category_mapping', 'classes', 'names']
                        for field in category_fields:
                            if field in data:
                                if isinstance(data[field], list):
                                    print(f"  ✅ {field}: {len(data[field])}개")
                                elif isinstance(data[field], dict):
                                    print(f"  ✅ {field}: {len(data[field])}개 (딕셔너리)")
                                    if field == 'category_mapping':
                                        original_ids = [int(k) for k in data[field].keys()]
                                        print(f"     원본 카테고리 ID 범위: {min(original_ids)} ~ {max(original_ids)}")
                    else:
                        # YAML 파일 간단 파싱
                        content = f.read()
                        if 'nc:' in content:
                            for line in content.split('\n'):
                                if 'nc:' in line:
                                    print(f"  ✅ YAML에서 클래스 수 발견: {line.strip()}")

            except Exception as e:
                print(f"  ❌ 파일 읽기 실패: {e}")
        else:
            print(f"  ❌ 파일 없음: {file_path}")

In [ ]:
if __name__ == "__main__":
    # 1단계 실행
    result = analyze_total_categories_from_annotations()

    # 대안 방법으로 검증
    verify_with_alternative_methods()

    print(f"\n{'='*50}")
    print("=== 1단계 완료 ===")
    if result:
        print(f"✅ 전체 카테고리 수: {result['total_categories']}개")
        if result['total_categories'] == 82:
            print("✅ 예상과 일치!")
        else:
            print(f"⚠️ 예상(82개)과 다름")

        if result['rare_categories']:
            print(f"⚠️ 희귀 카테고리: {len(result['rare_categories'])}개 발견")
            print("   (이들이 미탐지될 가능성이 높음)")
    else:
        print("❌ 카테고리 정보를 찾지 못했습니다.")
        print("   데이터 경로를 확인하거나 대안 방법을 시도하세요.")

=== 1단계: 전체 카테고리 82개 확인 ===
📁 어노테이션 폴더 탐색: ./data/train_annotations
  처리 중: 100/100 파일
  처리 중: 200/200 파일
  처리 중: 300/300 파일
  처리 중: 400/400 파일
  처리 중: 500/500 파일
  처리 중: 600/600 파일
  처리 중: 700/700 파일
  처리 중: 800/800 파일
  처리 중: 900/900 파일
  처리 중: 1000/1000 파일
  처리 중: 1100/1100 파일
  처리 중: 1200/1200 파일
  처리 중: 1300/1300 파일
  처리 중: 1400/1400 파일
  처리 중: 1500/1500 파일
  처리 중: 1600/1600 파일
  처리 중: 1700/1700 파일
  처리 중: 1800/1800 파일
  처리 중: 1900/1900 파일
  처리 중: 2000/2000 파일
  처리 중: 2100/2100 파일
  처리 중: 2200/2200 파일
  처리 중: 2300/2300 파일
  처리 중: 2400/2400 파일
  처리 중: 2500/2500 파일
  처리 중: 2600/2600 파일
  처리 중: 2700/2700 파일
  처리 중: 2800/2800 파일
  처리 중: 2900/2900 파일
  처리 중: 3000/3000 파일
  처리 중: 3100/3100 파일
  처리 중: 3200/3200 파일
  처리 중: 3300/3300 파일
  처리 중: 3400/3400 파일
  처리 중: 3500/3500 파일
  처리 중: 3600/3600 파일
  처리 중: 3700/3700 파일
  처리 중: 3800/3800 파일
  처리 중: 3900/3900 파일
  처리 중: 4000/4000 파일
  처리 중: 4100/4100 파일
  처리 중: 4200/4200 파일
  처리 중: 4300/4300 파일
  처리 중: 4400/4400 파일
  처리 중: 4500/4500 파일

=== 

In [ ]:
def analyze_missing_categories():
    """73개 vs 82개 차이 원인 분석"""
    print("=== 누락된 9개 카테고리 원인 분석 ===")

    annotations_root = "./data/train_annotations"

    # 1단계에서 발견된 73개 카테고리 로드
    try:
        with open('./total_categories_analysis.json', 'r', encoding='utf-8') as f:
            previous_result = json.load(f)
            found_categories = set(previous_result['category_ids'])
            print(f"✅ 1단계에서 발견된 카테고리: {len(found_categories)}개")
    except:
        print("❌ 1단계 결과 파일을 찾을 수 없습니다. 다시 분석합니다.")
        found_categories = set()

    # 상세 분석을 위한 변수들
    file_analysis = {
        'total_files': 0,
        'processed_files': 0,
        'empty_categories': 0,
        'no_categories_field': 0,
        'parse_errors': 0,
        'files_with_categories': []
    }

    all_categories_detailed = defaultdict(lambda: {
        'name': 'Unknown',
        'file_count': 0,
        'first_seen_file': None,
        'supercategory': None
    })

    print(f"\n📁 상세 파일 분석 시작...")

    # 모든 JSON 파일 상세 분석
    for root, dirs, files in os.walk(annotations_root):
        for file in files:
            if file.endswith('.json'):
                file_analysis['total_files'] += 1
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, annotations_root)

                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                    if 'categories' not in data:
                        file_analysis['no_categories_field'] += 1
                        continue

                    categories = data['categories']
                    if len(categories) == 0:
                        file_analysis['empty_categories'] += 1
                        continue

                    # 카테고리 정보 상세 수집
                    file_categories = []
                    for category in categories:
                        cat_id = category['id']
                        cat_name = category.get('name', f'category_{cat_id}')
                        supercategory = category.get('supercategory', 'pill')

                        # 상세 정보 업데이트
                        if all_categories_detailed[cat_id]['file_count'] == 0:
                            all_categories_detailed[cat_id]['first_seen_file'] = relative_path

                        all_categories_detailed[cat_id]['name'] = cat_name
                        all_categories_detailed[cat_id]['file_count'] += 1
                        all_categories_detailed[cat_id]['supercategory'] = supercategory

                        file_categories.append(cat_id)

                    file_analysis['files_with_categories'].append({
                        'file': relative_path,
                        'category_count': len(file_categories),
                        'categories': file_categories
                    })

                    file_analysis['processed_files'] += 1

                    if file_analysis['processed_files'] % 200 == 0:
                        print(f"  처리 중: {file_analysis['processed_files']}/{file_analysis['total_files']}")

                except json.JSONDecodeError as e:
                    file_analysis['parse_errors'] += 1
                    print(f"  ⚠️ JSON 파싱 오류: {relative_path}")
                except Exception as e:
                    print(f"  ⚠️ 기타 오류: {relative_path} - {e}")

    # 분석 결과 정리
    current_categories = set(all_categories_detailed.keys())

    print(f"\n=== 상세 분석 결과 ===")
    print(f"📄 전체 JSON 파일: {file_analysis['total_files']}개")
    print(f"✅ 성공적 처리: {file_analysis['processed_files']}개")
    print(f"❌ categories 필드 없음: {file_analysis['no_categories_field']}개")
    print(f"❌ 빈 categories: {file_analysis['empty_categories']}개")
    print(f"❌ 파싱 오류: {file_analysis['parse_errors']}개")
    print(f"🎯 현재 발견된 카테고리: {len(current_categories)}개")

    # 원인 1: 테스트 데이터에만 존재하는 카테고리 확인
    print(f"\n=== 원인 1: 테스트 결과에서 훈련 데이터에 없는 카테고리 찾기 ===")

    test_categories = set()
    result_files = [
        'submission_detailed.csv',
        'submission_thresh_0.323 1.csv'
    ]

    for file_name in result_files:
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)
            file_test_cats = set(df['category_id'].unique())
            test_categories.update(file_test_cats)
            print(f"  {file_name}: {len(file_test_cats)}개 카테고리")

    # 훈련 데이터에 없는 카테고리
    missing_in_train = test_categories - current_categories
    if missing_in_train:
        print(f"  ⚠️ 훈련 데이터에 없지만 테스트에서 탐지된 카테고리: {len(missing_in_train)}개")
        print(f"     카테고리 ID: {sorted(missing_in_train)}")
    else:
        print(f"  ✅ 모든 테스트 카테고리가 훈련 데이터에 존재")

    # 원인 2: 외부 소스와 비교
    print(f"\n=== 원인 2: 외부 매핑 파일과 비교 ===")

    mapping_files = [
        './data/yolo_format/category_id_mapping.json',
        './data/processed/category_id_mapping.json'
    ]

    external_categories = set()
    for mapping_file in mapping_files:
        if os.path.exists(mapping_file):
            with open(mapping_file, 'r', encoding='utf-8') as f:
                mapping_data = json.load(f)

            if 'category_mapping' in mapping_data:
                external_cats = set(int(k) for k in mapping_data['category_mapping'].keys())
                external_categories.update(external_cats)
                print(f"  ✅ {mapping_file}: {len(external_cats)}개 카테고리")

    if external_categories:
        missing_in_annotations = external_categories - current_categories
        if missing_in_annotations:
            print(f"  ⚠️ 매핑 파일에 있지만 어노테이션에 없는 카테고리: {len(missing_in_annotations)}개")
            print(f"     카테고리 ID: {sorted(missing_in_annotations)}")
        else:
            print(f"  ✅ 매핑 파일과 어노테이션이 일치")

    # 원인 3: 희귀 카테고리 분석
    print(f"\n=== 원인 3: 희귀 카테고리 분석 ===")

    category_frequency = {cat_id: info['file_count'] for cat_id, info in all_categories_detailed.items()}
    frequency_stats = Counter(category_frequency.values())

    print(f"카테고리 출현 빈도 분포:")
    for freq in sorted(frequency_stats.keys())[:10]:
        print(f"  {freq}회 출현: {frequency_stats[freq]}개 카테고리")

    # 매우 희귀한 카테고리들
    very_rare = [cat_id for cat_id, freq in category_frequency.items() if freq <= 2]
    if very_rare:
        print(f"\n매우 희귀한 카테고리 (2회 이하): {len(very_rare)}개")
        for cat_id in sorted(very_rare)[:5]:
            info = all_categories_detailed[cat_id]
            print(f"  ID {cat_id}: {info['name']} ({info['file_count']}회, 첫 발견: {info['first_seen_file']})")

    # 원인 4: 가설 검증
    print(f"\n=== 가설 검증 ===")

    total_found = len(current_categories)
    missing_count = 82 - total_found

    print(f"현재 상황: {total_found}개 발견, {missing_count}개 누락")

    # 가설들
    hypotheses = []

    if missing_in_train:
        hypotheses.append(f"가설 1: {len(missing_in_train)}개가 테스트 전용 카테고리")

    if external_categories and missing_in_annotations:
        hypotheses.append(f"가설 2: {len(missing_in_annotations)}개가 매핑 파일에만 존재")

    if file_analysis['no_categories_field'] > 0 or file_analysis['empty_categories'] > 0:
        problematic_files = file_analysis['no_categories_field'] + file_analysis['empty_categories']
        hypotheses.append(f"가설 3: {problematic_files}개 파일에 카테고리 정보 누락")

    hypotheses.append(f"가설 4: 82개라는 숫자가 잘못됨 (실제로는 {total_found}개)")

    print("가능한 원인들:")
    for i, hypothesis in enumerate(hypotheses, 1):
        print(f"  {i}. {hypothesis}")

    # 결과 저장
    detailed_result = {
        'analysis_type': 'missing_categories_investigation',
        'found_categories': total_found,
        'expected_categories': 82,
        'missing_count': missing_count,
        'file_analysis': file_analysis,
        'current_categories': sorted(current_categories),
        'missing_in_train': sorted(missing_in_train) if missing_in_train else [],
        'external_categories': sorted(external_categories) if external_categories else [],
        'very_rare_categories': sorted(very_rare) if very_rare else [],
        'hypotheses': hypotheses
    }

    with open('missing_categories_analysis.json', 'w', encoding='utf-8') as f:
        json.dump(detailed_result, f, ensure_ascii=False, indent=2)

    print(f"\n💾 상세 분석 결과 저장: missing_categories_analysis.json")

    return detailed_result

In [ ]:
def suggest_next_actions(analysis_result):
    """다음 단계 액션 제안"""
    print(f"\n{'='*50}")
    print("=== 권장 다음 단계 ===")

    missing_count = analysis_result['missing_count']
    missing_in_train = analysis_result['missing_in_train']

    if missing_in_train:
        print(f"🎯 우선순위 1: 테스트 전용 카테고리 {len(missing_in_train)}개 확인")
        print(f"   이들이 누락된 {missing_count}개의 일부일 가능성 높음")
        print(f"   → 이 카테고리들의 원본 데이터 위치 찾기")

    if analysis_result['very_rare_categories']:
        print(f"🎯 우선순위 2: 희귀 카테고리들이 실제로 유효한지 확인")
        print(f"   → 이미지 파일과 어노테이션 매칭 검증")

    if missing_count <= len(missing_in_train):
        print(f"✅ 결론: 82개는 전체 데이터셋 기준, 73개는 훈련 데이터 기준일 가능성")
        print(f"   → 2단계(미탐지 카테고리 분석)로 진행 가능")
    else:
        print(f"⚠️ 추가 조사 필요: 아직 {missing_count - len(missing_in_train)}개가 미해결")

In [ ]:
if __name__ == "__main__":
    result = analyze_missing_categories()
    suggest_next_actions(result)

=== 누락된 9개 카테고리 원인 분석 ===
✅ 1단계에서 발견된 카테고리: 73개

📁 상세 파일 분석 시작...
  처리 중: 200/200
  처리 중: 400/400
  처리 중: 600/600
  처리 중: 800/800
  처리 중: 1000/1000
  처리 중: 1200/1200
  처리 중: 1400/1400
  처리 중: 1600/1600
  처리 중: 1800/1800
  처리 중: 2000/2000
  처리 중: 2200/2200
  처리 중: 2400/2400
  처리 중: 2600/2600
  처리 중: 2800/2800
  처리 중: 3000/3000
  처리 중: 3200/3200
  처리 중: 3400/3400
  처리 중: 3600/3600
  처리 중: 3800/3800
  처리 중: 4000/4000
  처리 중: 4200/4200
  처리 중: 4400/4400

=== 상세 분석 결과 ===
📄 전체 JSON 파일: 4526개
✅ 성공적 처리: 4526개
❌ categories 필드 없음: 0개
❌ 빈 categories: 0개
❌ 파싱 오류: 0개
🎯 현재 발견된 카테고리: 73개

=== 원인 1: 테스트 결과에서 훈련 데이터에 없는 카테고리 찾기 ===
  ✅ 모든 테스트 카테고리가 훈련 데이터에 존재

=== 원인 2: 외부 매핑 파일과 비교 ===

=== 원인 3: 희귀 카테고리 분석 ===
카테고리 출현 빈도 분포:
  7회 출현: 1개 카테고리
  9회 출현: 2개 카테고리
  10회 출현: 1개 카테고리
  11회 출현: 1개 카테고리
  16회 출현: 2개 카테고리
  18회 출현: 1개 카테고리
  21회 출현: 1개 카테고리
  22회 출현: 1개 카테고리
  23회 출현: 4개 카테고리
  24회 출현: 2개 카테고리

=== 가설 검증 ===
현재 상황: 73개 발견, 9개 누락
가능한 원인들:
  1. 가설 4: 82개라는 숫자가 잘못됨 (실제로는 73개)

💾 상세 분석 결과 저장: miss

In [5]:
def calculate_file_hash(file_path, hash_algorithm='md5'):
    """
    파일의 해시값을 계산합니다.

    Args:
        file_path (str): 파일 경로
        hash_algorithm (str): 해시 알고리즘 ('md5', 'sha256' 등)

    Returns:
        str: 파일의 해시값
    """
    hash_func = hashlib.new(hash_algorithm)
    try:
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_func.update(chunk)
        return hash_func.hexdigest()
    except Exception as e:
        print(f"Error calculating hash for {file_path}: {e}")
        return None

In [6]:
def find_duplicate_images(train_images_dir):
    """
    train_images 폴더에서 중복된 이미지를 찾습니다.
    """
    print("🔍 이미지 중복 검사를 시작합니다...")

    # 파일명과 해시값을 저장할 딕셔너리
    filename_groups = defaultdict(list)
    hash_groups = defaultdict(list)

    # 지원하는 이미지 확장자
    image_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif'}

    # train_images 폴더의 모든 파일 검사
    train_path = Path(train_images_dir)
    if not train_path.exists():
        print(f"❌ 경로를 찾을 수 없습니다: {train_images_dir}")
        return {}

    image_files = []
    for file_path in train_path.iterdir():
        if file_path.is_file() and file_path.suffix.lower() in image_extensions:
            image_files.append(file_path)

    total_files = len(image_files)
    print(f"📊 총 {total_files}개의 이미지 파일을 발견했습니다.")

    if total_files == 0:
        print("❌ 이미지 파일이 없습니다.")
        return {}

    # 각 파일에 대해 정보 수집
    for i, file_path in enumerate(image_files):
        if (i + 1) % 100 == 0 or i == 0:
            print(f"진행률: {i+1}/{total_files} ({((i+1)/total_files)*100:.1f}%)")

        filename = file_path.name
        file_hash = calculate_file_hash(str(file_path))

        if file_hash:
            filename_groups[filename].append(str(file_path))
            hash_groups[file_hash].append(str(file_path))

    # 중복 파일 찾기
    filename_duplicates = {k: v for k, v in filename_groups.items() if len(v) > 1}
    hash_duplicates = {k: v for k, v in hash_groups.items() if len(v) > 1}

    # 결과 출력
    print("\n" + "="*50)
    print("📋 이미지 중복 검사 결과")
    print("="*50)

    print(f"\n🏷️  파일명 기준 중복:")
    if filename_duplicates:
        for filename, paths in filename_duplicates.items():
            print(f"  - {filename}: {len(paths)}개")
            for path in paths:
                print(f"    • {path}")
    else:
        print("  ✅ 파일명 중복 없음")

    print(f"\n🔒 해시값 기준 중복 (실제 내용 동일):")
    if hash_duplicates:
        for file_hash, paths in hash_duplicates.items():
            print(f"  - Hash {file_hash[:8]}...: {len(paths)}개")
            for path in paths:
                print(f"    • {path}")
    else:
        print("  ✅ 해시값 중복 없음")

    return {
        'filename_duplicates': filename_duplicates,
        'hash_duplicates': hash_duplicates,
        'total_files': total_files
    }

In [7]:
def find_duplicate_annotations(train_annotations_dir):
    """
    train_annotations 폴더에서 중복된 어노테이션을 찾습니다.

    Args:
        train_annotations_dir (str): train_annotations 폴더 경로

    Returns:
        dict: 중복 정보가 담긴 딕셔너리
    """
    print("\n🔍 어노테이션 중복 검사를 시작합니다...")

    filename_groups = defaultdict(list)
    hash_groups = defaultdict(list)
    content_groups = defaultdict(list)

    # train_annotations 폴더의 모든 JSON 파일 검사
    annotations_path = Path(train_annotations_dir)
    if not annotations_path.exists():
        print(f"❌ 경로를 찾을 수 없습니다: {train_annotations_dir}")
        return {}

    json_files = []
    # 재귀적으로 모든 JSON 파일 찾기
    for json_file in annotations_path.rglob("*.json"):
        json_files.append(json_file)

    total_files = len(json_files)
    print(f"📊 총 {total_files}개의 JSON 파일을 발견했습니다.")

    if total_files == 0:
        print("❌ JSON 파일이 없습니다.")
        return {}

    # 각 파일에 대해 정보 수집
    for i, file_path in enumerate(json_files):
        if (i + 1) % 50 == 0 or i == 0:
            print(f"진행률: {i+1}/{total_files} ({((i+1)/total_files)*100:.1f}%)")

        filename = file_path.name
        file_hash = calculate_file_hash(str(file_path))

        # JSON 내용도 비교 (정렬된 형태로)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                json_content = json.load(f)
            # 정렬된 JSON 문자열로 변환하여 내용 비교
            content_str = json.dumps(json_content, sort_keys=True, ensure_ascii=False)
            content_hash = hashlib.md5(content_str.encode()).hexdigest()

            filename_groups[filename].append(str(file_path))
            if file_hash:
                hash_groups[file_hash].append(str(file_path))
            content_groups[content_hash].append(str(file_path))

        except Exception as e:
            print(f"⚠️  JSON 파일 읽기 실패 {file_path}: {e}")

    # 중복 파일 찾기
    filename_duplicates = {k: v for k, v in filename_groups.items() if len(v) > 1}
    hash_duplicates = {k: v for k, v in hash_groups.items() if len(v) > 1}
    content_duplicates = {k: v for k, v in content_groups.items() if len(v) > 1}

    # 결과 출력
    print("\n" + "="*50)
    print("📋 어노테이션 중복 검사 결과")
    print("="*50)

    print(f"\n🏷️  파일명 기준 중복:")
    if filename_duplicates:
        for filename, paths in filename_duplicates.items():
            print(f"  - {filename}: {len(paths)}개")
            for path in paths:
                print(f"    • {path}")
    else:
        print("  ✅ 파일명 중복 없음")

    print(f"\n🔒 파일 해시값 기준 중복:")
    if hash_duplicates:
        for file_hash, paths in hash_duplicates.items():
            print(f"  - Hash {file_hash[:8]}...: {len(paths)}개")
            for path in paths:
                print(f"    • {path}")
    else:
        print("  ✅ 파일 해시값 중복 없음")

    print(f"\n📄 JSON 내용 기준 중복:")
    if content_duplicates:
        for content_hash, paths in content_duplicates.items():
            print(f"  - Content Hash {content_hash[:8]}...: {len(paths)}개")
            for path in paths:
                print(f"    • {path}")
    else:
        print("  ✅ JSON 내용 중복 없음")

    return {
        'filename_duplicates': filename_duplicates,
        'hash_duplicates': hash_duplicates,
        'content_duplicates': content_duplicates,
        'total_files': total_files
    }

In [8]:
def check_dataset_duplicates(data_dir):
    """
    전체 데이터셋의 중복을 검사하는 메인 함수

    Args:
        data_dir (str): 데이터셋 루트 디렉토리 경로

    Returns:
        dict: 전체 중복 검사 결과
    """
    print("🚀 데이터셋 중복 검사를 시작합니다!")
    print(f"📁 데이터 디렉토리: {data_dir}")

    # 경로 설정
    train_images_dir = os.path.join(data_dir, 'train_images')
    train_annotations_dir = os.path.join(data_dir, 'train_annotations')

    # 이미지 중복 검사
    image_results = find_duplicate_images(train_images_dir)

    # 어노테이션 중복 검사
    annotation_results = find_duplicate_annotations(train_annotations_dir)

    # 전체 요약
    print("\n" + "="*60)
    print("📊 전체 중복 검사 요약")
    print("="*60)

    if image_results:
        img_filename_dups = len(image_results.get('filename_duplicates', {}))
        img_hash_dups = len(image_results.get('hash_duplicates', {}))
        total_images = image_results.get('total_files', 0)
        print(f"🖼️  이미지: {total_images}개 중 파일명 중복 {img_filename_dups}개, 해시 중복 {img_hash_dups}개")

    if annotation_results:
        ann_filename_dups = len(annotation_results.get('filename_duplicates', {}))
        ann_hash_dups = len(annotation_results.get('hash_duplicates', {}))
        ann_content_dups = len(annotation_results.get('content_duplicates', {}))
        total_annotations = annotation_results.get('total_files', 0)
        print(f"📝 어노테이션: {total_annotations}개 중 파일명 중복 {ann_filename_dups}개, 파일 해시 중복 {ann_hash_dups}개, 내용 중복 {ann_content_dups}개")

    return {
        'image_results': image_results,
        'annotation_results': annotation_results
    }

In [9]:
def run_duplicate_check():
    """
    코랩에서 실행할 수 있는 예시 함수
    """
    # 데이터 디렉토리 경로를 설정하세요
    DATA_DIR = "./data"  # 또는 "/content/drive/MyDrive/your_dataset_path"

    # 중복 검사 실행
    results = check_dataset_duplicates(DATA_DIR)

    return results

In [10]:
results = run_duplicate_check()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
    • data/train_annotations/K-001900-016551-033009-044199_json/K-016551/K-001900-016551-033009-044199_0_2_0_2_90_000_200.json
    • data/train_annotations/K-001900-016551-033009-044199_json/K-033009/K-001900-016551-033009-044199_0_2_0_2_90_000_200.json
    • data/train_annotations/K-001900-016551-033009-044199_json/K-044199/K-001900-016551-033009-044199_0_2_0_2_90_000_200.json
  - K-001900-016551-033009-044199_0_2_0_2_70_000_200.json: 3개
    • data/train_annotations/K-001900-016551-033009-044199_json/K-001900/K-001900-016551-033009-044199_0_2_0_2_70_000_200.json
    • data/train_annotations/K-001900-016551-033009-044199_json/K-016551/K-001900-016551-033009-044199_0_2_0_2_70_000_200.json
    • data/train_annotations/K-001900-016551-033009-044199_json/K-044199/K-001900-016551-033009-044199_0_2_0_2_70_000_200.json
  - K-002483-003743-004378-006192_0_2_0_2_90_000_200.json: 2개
    • data/train_annotations/K-002483-003743-004378-006192_json/K-002483/K-002

============================================================            
📊 전체 중복 검사 요약          
============================================================
🖼️  이미지: 4498개 중 파일명 중복 0개, 해시 중복 0개            
📝 어노테이션: 16269개 중 파일명 중복 4431개, 파일 해시 중복 0개, 내용 중복 0개